# Offer Allocation Problem

Price discrimination is often achieved by having one stated public price but offer- ing individual consumers discounts or coupons which lower the individual consumer’s price. Finding the op- timal mix of discounts or coupons in order to maximize profitability is an optimal-selection problem, naturally formulated as a constrained quadratic binary model.

Assign $m$ offers to $n$ consumers. <br>
Each offer $m$ can only be given to a small number of consumers $c_{j}$, <br>
And each consumer can only receive a handful of offers $c_{i}$. 
Binary variable $a_{i, j}$ is assigned to each offer-consumer pair. 

**Constraint 1:** <br>
Constraint on the number of consumers given an offer : <br>
$\sum_{i}^{n} a_{i, j} \leq c_{j}$


**Constraint 2:** <br>
Constraint on the number of offers given to a consumer: <br>
$\sum_{i}^{n} a_{i, j} \leq c_{i}$


In [5]:
import dimod

In [7]:
def offer_allocation(m, n, c_consumers, c_offers, values):
    """Offer Allocation Function
    m: number of offers
    n: number of consumers
    c_consumers: total number of offers per consumer
    total_offers: total number of offers
    """

    cqm = dimod.ConstrainedQuadraticModel()

    assignments = {}
    for i in range(n): 
        # loops over number of consumers (n)
        for j in range(m):
            # loops of number of offers (m)
            # each offer-consumer pair is a binmary variable
            pair = dimod.Binary(f"consumer_{i}_offer_{j}")
            assignments[i, j] = pair
    
    # Constraint 2: each consumer gets c_consumers offers
    for i in range(n):
        cqm.add_constrains_from_comparison(
            dimod.quicksum(assignemnts[i, j] for j in range(m)
            ) <= c_consumers[i]
        )

    # Constraint 1: each offer can only be given a limited number of times
    for j in range(m):
        cqm.add_constraint_from_comparison(
            dimod.quicksum(assignments[i, j] for i in range(n)
            ) <= c_offers[j]
        )
    
    # Objective: Maximize total value of deal mix per customer offered deals
    cqm.set_objective(
        dimod.quicksum(
            -val * assignemnts[i, j] * assignments[i, k]
            for (i, j, k), val in values.items()
            if j > k
        ),
    )

    return cqm

In [8]:
import unittest
import numpy as np

class TestOfferAllocation(unittest.TestCase):
    def setUp(self):
        self.m = 5  # number of offers
        self.n = 10  # number of consumers
        self.c_consumers = np.random.randint(1, self.m, self.n)  # random number of offers per consumer
        self.c_offers = np.random.randint(1, self.n, self.m)  # random number of times each offer can be given
        self.values = {(i, j, k): np.random.rand() for i in range(self.n) for j in range(self.m) for k in range(self.m) if j > k}  # random values for each offer-consumer pair

    def test_offer_allocation(self):
        cqm = offer_allocation(self.m, self.n, self.c_consumers, self.c_offers, self.values)
        self.assertIsInstance(cqm, dimod.ConstrainedQuadraticModel)

if __name__ == "__main__":
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/home/samyaknj/.local/share/jupyter/runtime/kernel-v2-735942zGrwGlDL7C0f.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'